In [1]:
import re
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
BASE_RSPP_URL = "https://rspp.ru"
RSPP_URL_REGISTR = f"{BASE_RSPP_URL}/activity/social/registr/"
REPORT_TYPES = {
    "ОУР": "Отчет по устойчивому развитию",
    "СО": "Социальный отчет",
    "ИО": "Интегрированный отчет",
    "ЭО": "Экологический отчёт",
}
RE_YEARS = re.compile("....")
FOLDER_TO_SAVE_REPORTS = "reports"

In [3]:
def parse_td(company_report_row):
    report_info = [td.find("div") for td in company_report_row.findAll("td", class_="register-table__colclass")]
    company_name = report_info[0].find("span").text
    sector = report_info[1].text
    years_list = RE_YEARS.findall(report_info[2].text)
    years = ", ".join(years_list)
    report_tag_a = report_info[3].find("a")
    report_link, report_type = report_tag_a["href"], report_tag_a.text.strip()
    return {
        "компания": company_name,
        "сектор": sector,
        "год": years,
        "тип отчета": report_type,
        "ссылка на отчет": report_link,
    }

In [4]:
def parse_rspp():
    rspp_response = requests.get(RSPP_URL_REGISTR)
    assert rspp_response.status_code < 400, "Не получилось подключиться к сайту РСПП"
    soup = BeautifulSoup(rspp_response.text, "lxml")
    company_report_filter = soup.find("div", class_="company-report-filter")
    report_table = company_report_filter.find("table", class_="table_scroll-mobile")
    headline, *company_reports_list = report_table.findAll("tr", class_="register-table__row")
    company_reports_dicts = [parse_td(company_report_td) for company_report_td in tqdm(company_reports_list)]
    return pd.DataFrame.from_dict(company_reports_dicts)

In [12]:
# rspp_df = parse_rspp()
rspp_df = pd.read_csv("raex_esg_with_rspp.csv", index_col=0)
rspp_df = rspp_df[rspp_df["ссылка на отчет"].notna()]
rspp_df["id"] = rspp_df["id"].astype(int)
rspp_df.head()

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,...,Год последней оцененной отчетности,E_transformed,S_transformed,G_transformed,id,компания,сектор,год,тип отчета,ссылка на отчет
0.0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,...,2021.0,2.0,2.0,3.0,714,ПАО «НЛМК»,Металлургическая и горнодобывающая,2021.0,ИО,/download/de45f8ddf53aa9ba80614d1c93e296fc/
1.0,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,...,2021.0,1.0,3.0,3.0,887,ПАО «Полюс»,Металлургическая и горнодобывающая,2021.0,ОУР,/download/a79b0f48c2f54dc885ddf583ef4c402d/
2.0,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,...,2021.0,4.0,2.0,2.0,1275,ПАО «Уралкалий»,"Химическая, нефтехимическая, парфюмерная",2021.0,ОУР,/download/0e88d52331c02d9939be212b2b95e562/
4.0,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,...,2021.0,3.0,3.0,2.0,875,ПАО «Полиметалл»,Металлургическая и горнодобывающая,2021.0,ИО,/download/dd60bd2b772498057452e9b81e540671/
5.0,6,«ФосАгро»,PHOR,Агрохимикаты,A,8,BBB,11,A,5,...,2021.0,4.0,3.0,2.0,1353,ПАО «ФосАгро»,"Химическая, нефтехимическая, парфюмерная",2021.0,ИО,/download/148f63c45d44e827ae38173e7add3c83/


In [7]:
rspp_df.to_csv("rspp_reports.csv")

In [8]:
CHUNK_SIZE = 400


def download_report_pdf(link, filename, folder_to_save=FOLDER_TO_SAVE_REPORTS):
    download_url = f"{BASE_RSPP_URL}{link}"
    report_response = requests.get(download_url, stream=True)
    assert report_response.status_code == 200, "Не получилось скачать отчет с сайта РСПП"
    filename = f"{folder_to_save}/{filename}"
    with open(filename, "wb") as f:
        for chunk in report_response.iter_content(CHUNK_SIZE):
            f.write(chunk)

In [14]:
def download_reports_pdf(rspp_df):
    for index, report in tqdm(rspp_df.iterrows(), total=rspp_df.shape[0]):
        link = report["ссылка на отчет"]
        company = report["компания"].replace("/", "_")
        idx = report["id"]
        filename = f"{company}_{idx}.pdf"
        try:
            download_report_pdf(link, filename)
        except Exception as e:
            print(e, BASE_RSPP_URL + link)
        time.sleep(2)

In [15]:
import os

if not os.path.exists(FOLDER_TO_SAVE_REPORTS):
    os.makedirs(FOLDER_TO_SAVE_REPORTS)

In [17]:
download_reports_pdf(rspp_df)

 25%|██▌       | 13/51 [01:25<03:56,  6.22s/it]

HTTPSConnectionPool(host='rspp.ru', port=443): Max retries exceeded with url: /download/257dd0099122b60e4d5579814798863b/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcc789a6b60>: Failed to establish a new connection: [Errno -2] Name or service not known')) https://rspp.ru/download/257dd0099122b60e4d5579814798863b/


 43%|████▎     | 22/51 [02:16<02:50,  5.88s/it]

HTTPSConnectionPool(host='rspp.ru', port=443): Max retries exceeded with url: /download/3b7a670ea710f80b22fca17904d1ce72/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcc789a6a40>: Failed to establish a new connection: [Errno -2] Name or service not known')) https://rspp.ru/download/3b7a670ea710f80b22fca17904d1ce72/


 45%|████▌     | 23/51 [02:20<02:30,  5.39s/it]

HTTPSConnectionPool(host='rspp.ru', port=443): Max retries exceeded with url: /download/aa86885bccaefd9b26d62cfcfadd9b42/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fcc789a6a40>: Failed to establish a new connection: [Errno -2] Name or service not known')) https://rspp.ru/download/aa86885bccaefd9b26d62cfcfadd9b42/


100%|██████████| 51/51 [05:00<00:00,  5.89s/it]


# Parse per paragraph

In [22]:
import os
import re
from collections import defaultdict

import textract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from PyPDF2 import PdfReader, PdfWriter
from tqdm.autonotebook import tqdm
import pandas as pd

/tmp/ipykernel_35009/945261966.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [23]:
REPORTS_DIRECTORY = "reports"

RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile(r"[а-яА-Яa-zA-Z0-9\-]+")

stopwords_ru = stopwords.words("russian")

In [24]:
morph = MorphAnalyzer()
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}

In [25]:
info_df = pd.read_csv("rspp_reports.csv", index_col=0)
info_df.head()

,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,...,Год последней оцененной отчетности,E_transformed,S_transformed,G_transformed,id,компания,сектор,год,тип отчета,ссылка на отчет
0.0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,...,2021.0,2.0,2.0,3.0,714.0,ПАО «НЛМК»,Металлургическая и горнодобывающая,2021.0,ИО,/download/de45f8ddf53aa9ba80614d1c93e296fc/
1.0,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,...,2021.0,1.0,3.0,3.0,887.0,ПАО «Полюс»,Металлургическая и горнодобывающая,2021.0,ОУР,/download/a79b0f48c2f54dc885ddf583ef4c402d/
2.0,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,...,2021.0,4.0,2.0,2.0,1275.0,ПАО «Уралкалий»,"Химическая, нефтехимическая, парфюмерная",2021.0,ОУР,/download/0e88d52331c02d9939be212b2b95e562/
4.0,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,...,2021.0,3.0,3.0,2.0,875.0,ПАО «Полиметалл»,Металлургическая и горнодобывающая,2021.0,ИО,/download/dd60bd2b772498057452e9b81e540671/
5.0,6,«ФосАгро»,PHOR,Агрохимикаты,A,8,BBB,11,A,5,...,2021.0,4.0,3.0,2.0,1353.0,ПАО «ФосАгро»,"Химическая, нефтехимическая, парфюмерная",2021.0,ИО,/download/148f63c45d44e827ae38173e7add3c83/


In [26]:
files = [f"{REPORTS_DIRECTORY}/{file}" for file in os.listdir(REPORTS_DIRECTORY)]

In [31]:
def parse_pages_to_texts(files: list[str]):
    errors = []
    df = []
    for report_name in tqdm(files):
        if not report_name.endswith(".pdf"):
            continue
        try:
            report = textract.process(report_name)
        except Exception as e:
            errors.append((report_name, e))
            continue
        company_name = report_name[:-4].split("_")[0]
        report_idx = int(report_name[:-4].split("_")[-1])
        report_index = info_df[info_df["id"] == report_idx].index[0]
        for i, paragraph in enumerate(report.decode().split("\n\n")):
            paragraph = paragraph.replace("\n", " ").lower()
            report_page = RE_DERIVED.sub("", paragraph)
            report_page_lst = word_tokenize(report_page)
            # russian_report_page_lst = [w for w in filter(RE_RUSSIAN_TEXT.match, report_page_lst)]
            tokens = []
            # nouns = set()
            # verbs = set()
            # adj = set()
            for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
                if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                    word_ = word_.strip()
                    word_ = morph.parse(word_)[0]
                    normal_form = word_.normal_form
                    tokens.append(normal_form)
                    # if word_.tag.POS in NOUNS:
                    #     nouns.add(normal_form)
                    # elif word_.tag.POS in VERBS:
                    #     verbs.add(normal_form)
                    # elif word_.tag.POS in ADJ:
                    #     adj.add(normal_form)
            # if len(tokens) == 0:
            #     continue

            df.append(
                {
                    "rsspp_index": report_idx,
                    "index": report_index,
                    "company": company_name,
                    "year": info_df.at[report_index, "год"],
                    "sector": info_df.at[report_index, "сектор"],
                    "report_type": info_df.at[report_index, "тип отчета"],
                    "paragraph": i,
                    "original_text": paragraph,
                    "cleaned_text": " ".join(tokens),
                }
            )
    print(*errors, sep="\n")
    return df

In [32]:
df = parse_pages_to_texts(files)

  0%|          | 0/51 [00:00<?, ?it/s]

In [33]:
new_df = pd.DataFrame(df)
new_df.head()

,rsspp_index,index,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,674,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,0,та новая выс,тот новый выс
1,674,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,1,взаимодействия,взаимодействие
2,674,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,2,годовой отчет за 2020 год,годовой отчёт 2020 год
3,674,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,3,содержание,содержание
4,674,65.0,reports/ПАО «МегаФон»,2020.0,Телекоммуникационная и связь,ИО,4,2,2


In [ ]:
new_df.to_csv("paragraphs.csv.zip")